<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Onshape_Flyover_GIF_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import and Setup Onshape Client

!pip install onshape-client
from onshape_client.client import Client
from onshape_client.onshape_url import OnshapeElement
import json

#@markdown Chage the base if using an enterprise (i.e. "https://ptc.onshape.com")
base = 'https://cad.onshape.com' #@param {type:"string"}

#@markdown Would you like to import your API keys from a file, or copy and paste them directly?
keyImportOption = "Upload Keys from File" #@param ["Upload Keys from File", "Copy/Paste Keys"]

from IPython.display import clear_output 
clear_output()
print("Onshape Client successfully imported!")

if keyImportOption == "Upload Keys from File":
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    execfile(fn)

  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')
else:
  access = input("Paste your Onshape Access Key: ")
  secret = input("Paste your Onshape Secret Key: ")
  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')


Onshape client configured - ready to go!


## Define image functions

In [24]:
#@title Get Shaded View of Assembly
#@markdown Defines funciton `assembliesShadedView(url:str,viewMatrix = "front",pixelSize = 0.003,edges = "show",filename = "image.jpg")`, which returns the base64 image data of an assembly and saves the image as a jpeg called "image.jpg"

#@markdown URL must be to an assembly
url = 'https://cad.onshape.com/documents/fb00ed566c3e0934ae91d74c/w/80749fc9b1aa0adab776dc56/e/f17ce18736063f03b8d7dd9a' #@param {type:"string"}
#@markdown viewMatrix can be any face direction (top, front, right, etc.) or isometric as a string, or a 1x12 view matrix
viewMatrix = "isometric" #@param {type:"string"}
#@markdown pixelSize is the size in meters for each pixel. If 0, it will fill the image size output
pixelSize = 0 #@param {type:"number"}
#@markdown You can set edges of your models to "show" or "hide"
edges = "hide" #@param {type:"string"}
# showImage = True #@param {type:"boolean"}

from IPython.display import Image
import base64

def assembliesShadedView(url:str,viewMatrix = "front",pixelSize = 0.003,edges = "show",filename = "image.jpg",outputHeight = 600,outputWidth = 1000):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/shadedviews'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  if any(face in viewMatrix for face in ["front","back","top","bottom","left","right"]):
    matrix = viewMatrix
  elif viewMatrix == "isometric":
    matrix = "0.612,0.612,0,0,-0.354,0.354,0.707,0,0.707,-0.707,0.707,0"
  elif isinstance(viewMatrix,list):
    matrix = str(viewMatrix).replace('[','').replace(']','')

  ## View Matrix below is roughly isometric
  params = {'viewMatrix':matrix,
            'edges':edges,
            'outputHeight':outputHeight,
            'outputWidth':outputWidth,
            'pixelSize':pixelSize}
  # print(params)
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
              'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  imgdata = base64.b64decode(parsed['images'][0])
  with open(filename, 'wb') as f:
    f.write(imgdata)
  
  return imgdata

img = assembliesShadedView(url,viewMatrix,pixelSize,edges)
Image(img)

## Additional imports and function for generating GIF

from PIL import Image as Image2
import time
import imageio
import io
import numpy as np
import math

def gen_frame(path):
    im = Image2.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image2.ADAPTIVE, colors=255)

    # Set all pixel values below 5 to 255 , and the rest to 0
    mask = Image2.eval(alpha, lambda a: 255 if a <=5 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(255, mask)

    # The transparency index is 255
    im.info['transparency'] = 255

    return im


### Optional: get shaded views from different perspectives
Run the cells below to define functions that manipulate the "view matrix" used by Onshape to specify where the shaded view will be captured from.

In [22]:
#@title View Matrix Helper Functions
#@markdown Run this cell to load a library of functions for view matrices

import math
import numpy

#@markdown The function `mult(x,y)` multiplies two 4x3 view matrices to get their determinant

def mult(x,y):
  result = numpy.matmul(x,y)
  return result

#@markdown The function `IdentityTwelve()` returns a flattened identity view matrix (1x12)
def identityTwelve():
  m = [
      1, 0, 0, 0,
      0, 1, 0, 0,
      0, 0, 1, 0
      ]
  return m

#@markdown The function `Move(base,x1,y1,z1)` takes a 1x12 view matrix and moves the x,y,z coordinates
def moveMatrix(base,x1,y1,z1):
  matrix = base
  matrix[0][3] = x1
  matrix[1][3] = y1
  matrix[2][3] = z1
  return matrix

#@markdown The function `Move(base,x1,y1,z1)` takes a 1x12 view matrix and moves the x,y,z coordinates
def moveFlat(base,x1,y1,z1):
  matrix = base
  matrix[3] = x1
  matrix[7] = y1
  matrix[11] = z1
  return matrix

#@markdown The function `twelveToThreeByFour(matrix)` takes a flattened 1x12 view matrix and makes a 4x3 matrix for linear algrebra
def twelveToThreeByFour(matrix):
  threebyfour = [[matrix[0],matrix[1],matrix[2],matrix[3]],
                [matrix[4],matrix[5],matrix[6],matrix[7]],
                [matrix[8],matrix[9],matrix[10],matrix[11]]]
  return threebyfour

#@markdown The function `threeByFourToTwelve(matrix)` takes a 4x3 view matrix and flattens it to 1x12, the form used by Onshape
def threeByFourToTwelve(matrix):
  twelve = [matrix[0][0],matrix[0][1],matrix[0][2],matrix[0][3],
             matrix[1][0],matrix[1][1],matrix[1][2],matrix[1][3],
             matrix[2][0],matrix[2][1],matrix[2][2],matrix[2][3]]
  return twelve

def fourByFourToThreeByFour(matrix):
  matrix.pop(3)
  return matrix

#@markdown The functions `clockwiseSpinX(theta)`, `clockwiseSpinY(theta)`, and `clockwiseSpinZ(theta)` returns a 4x3 matrix with a rotation of theta around the specified axis.
def clockwiseSpinX(theta):
  m = [[1, 0, 0, 0],
       [0, math.cos(theta), math.sin(theta), 0],
       [0, -math.sin(theta), math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinY(theta):
  m = [[math.cos(theta), 0, math.sin(theta), 0],
       [0, 1, 0, 0],
       [-math.sin(theta), 0, math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinZ(theta):
  m = [[math.cos(theta), math.sin(theta), 0, 0],
       [-math.sin(theta), math.cos(theta), 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]]
  return m


# View Matrix Key Frames


In [ ]:
#@title Get named views from Assembly
#@markdown Function `assembliesNamedViews(url: str)` returns JSON of all named views in an assembly
url = 'https://cad.onshape.com/documents/d1448e1a9a0ba70defc593b2/w/fbb3bdfdea457112f17b6a56/e/913330ba7e114b144ac9a8c1' #@param {type:"string"}
showResponse = False #@param {type:"boolean"}

from IPython.display import Image
import base64

def assembliesNamedViews(url:str):
  fixed_url = '/api/assemblies/d/did/e/eid/namedViews'
  element = OnshapeElement(url)
  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('eid', element.eid)

  method = 'GET'

  ## View Matrix below is roughly isometric
  params = {}
  # print(params)
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
              'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  
  return parsed

if showResponse:
  views = assembliesNamedViews(url)
  print(json.dumps(views, indent=4, sort_keys=True))
else:
  pass

# Stepping rotation

In [25]:
startingNamedView = 'view 1' #@param {type:'string'}
totalZRotationAngle = math.pi/4 #@param
numberOfFrames = 60 #@param
zoomStart = 0.001 #@param
zoomEnd = 0.0005 #@param
translationStart = [0,0,0] #@param
translationEnd = [0,0,-0.05] #@param
url = 'https://cad.onshape.com/documents/d1448e1a9a0ba70defc593b2/w/fbb3bdfdea457112f17b6a56/e/913330ba7e114b144ac9a8c1' #@param {type:"string"}

viewMatrices = assembliesNamedViews(url)
viewArray = viewMatrices['namedViews'][startingNamedView]['viewMatrix']
origArray = np.array([viewArray[0:4],viewArray[4:8],viewArray[8:12],viewArray[12:16]])

# Build new array from old array

newArray = [viewArray[0:4],viewArray[4:8],viewArray[8:12]]
newArrayFlat = viewArray[0:4]+viewArray[4:8]+viewArray[8:12]
# newArray = [viewArray[0:3]+[0],viewArray[4:7]+[0],viewArray[8:11]+[0]]
# newArrayFlat = viewArray[0:3]+[0]+viewArray[4:7]+[0]+viewArray[8:11]+[4]

images = []

matrix = newArray

translationArray = np.linspace(translationStart,translationEnd,numberOfFrames)
zoomArray = np.linspace(zoomStart,zoomEnd,numberOfFrames)

for i in range(numberOfFrames):
  matrix = mult(matrix,clockwiseSpinZ(totalZRotationAngle/numberOfFrames))
  matrix = moveMatrix(matrix,translationArray[i][0],translationArray[i][1],translationArray[i][2])
  if i == 0:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,zoomArray[i],"hide","image.jpg")
    im1 = gen_frame("image.jpg")
  else:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,zoomArray[i],"hide","image.jpg")
    images.append(gen_frame("image.jpg"))

im1.save('OnshapeGIF.gif', save_all=True, loop = 500, append_images=images, disposal=2, duration=200)

# Linear interpolation attempt 1

In [ ]:
namedView1 = 'view 1' #@param {type:'string'}
namedView2 = 'view 3' #@param {type:'string'}

viewMatrices = assembliesNamedViews(url)
viewArray1 = viewMatrices['namedViews'][namedView1]['viewMatrix']
origArray1 = np.array([viewArray1[0:4],viewArray1[4:8],viewArray1[8:12],viewArray1[12:16]])

viewArray2 = viewMatrices['namedViews'][namedView2]['viewMatrix']
origArray2 = np.array([viewArray2[0:4],viewArray2[4:8],viewArray2[8:12],viewArray2[12:16]])
# print("Original Named View Matrix")
# print(origArray1)
# print("")

## Try to create 4x3 view matrix for the shaded views api (which takes a 1x12 flattened array) to get the same image

# First transpose the original array

# origTrans1 = np.transpose(origArray1)
# origTrans2 = np.transpose(origArray1)
# print("Transposed Named View Matrix")
# print(origTrans1)
# print("")

# Build new array from old array

newArray1 = [viewArray1[0:4],viewArray1[4:8],viewArray1[8:12]]
newArrayFlat1 = viewArray1[0:4]+viewArray1[4:8]+viewArray1[8:12]
# newArray1 = [viewArray1[0:3]+[0],viewArray1[4:7]+[0],viewArray1[8:11]+[0]]
# newArrayFlat1 = viewArray1[0:3]+[0]+viewArray1[4:7]+[0]+viewArray1[8:11]+[4]

newArray2 = [viewArray2[0:4],viewArray2[4:8],viewArray2[8:12]]
newArrayFlat2 = viewArray2[0:4]+viewArray2[4:8]+viewArray2[8:12]
# newArray2 = [viewArray2[0:3]+[0],viewArray2[4:7]+[0],viewArray2[8:11]+[0]]
# newArrayFlat2 = viewArray2[0:3]+[3]+viewArray2[4:7]+[3]+viewArray2[8:11]+[4]

arrayArray = np.linspace(newArray1,newArray2,25)
print(arrayArray)

# print(np.array(newArray1))
# print(np.transpose(np.array(newArray)))
# flattenedArrayArray = arrayArray[0][0][0:4].tolist()+arrayArray[0][1][0:4].tolist()+arrayArray[0][2][0:4].tolist()
# print(flattenedArrayArray)
# img = assembliesShadedView(url,flattenedArrayArray,0.001)
# display(Image(img))

images = []

for ind, matrix in enumerate(arrayArray):
  if ind == 0:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,0.001,"hide","image.jpg")
    im1 = gen_frame("image.jpg")
  else:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,0.001,"hide","image.jpg")
    images.append(gen_frame("image.jpg"))

im1.save('OnshapeGIF.gif', save_all=True, loop = 500, append_images=images, disposal=2, duration=200)

# Linear interpolation attempt 2

In [ ]:
namedView1 = 'view 1' #@param {type:'string'}
namedView2 = 'view 3' #@param {type:'string'}

viewMatrices = assembliesNamedViews(url)
viewArray1 = viewMatrices['namedViews'][namedView1]['viewMatrix']
origArray1 = np.array([viewArray1[0:4],viewArray1[4:8],viewArray1[8:12],viewArray1[12:16]])

viewArray2 = viewMatrices['namedViews'][namedView2]['viewMatrix']
origArray2 = np.array([viewArray2[0:4],viewArray2[4:8],viewArray2[8:12],viewArray2[12:16]])
# print("Original Named View Matrix")
# print(origArray1)
# print("")

## Try to create 4x3 view matrix for the shaded views api (which takes a 1x12 flattened array) to get the same image

# First transpose the original array

# origTrans1 = np.transpose(origArray1)
# origTrans2 = np.transpose(origArray1)
# print("Transposed Named View Matrix")
# print(origTrans1)
# print("")

# Build new array from old array

newArray1 = [viewArray1[0:4],viewArray1[4:8],viewArray1[8:12]]
newArrayFlat1 = viewArray1[0:4]+viewArray1[4:8]+viewArray1[8:12]
# newArray1 = [viewArray1[0:3]+[0],viewArray1[4:7]+[0],viewArray1[8:11]+[0]]
# newArrayFlat1 = viewArray1[0:3]+[0]+viewArray1[4:7]+[0]+viewArray1[8:11]+[4]

newArray2 = [viewArray2[0:4],viewArray2[4:8],viewArray2[8:12]]
newArrayFlat2 = viewArray2[0:4]+viewArray2[4:8]+viewArray2[8:12]
# newArray2 = [viewArray2[0:3]+[0],viewArray2[4:7]+[0],viewArray2[8:11]+[0]]
# newArrayFlat2 = viewArray2[0:3]+[3]+viewArray2[4:7]+[3]+viewArray2[8:11]+[4]

arrayArray = np.linspace(origArray1,origArray2,10)
print(arrayArray)

# print(np.array(newArray1))
# print(np.transpose(np.array(newArray)))
# flattenedArrayArray = arrayArray[0][0][0:4].tolist()+arrayArray[0][1][0:4].tolist()+arrayArray[0][2][0:4].tolist()
# print(flattenedArrayArray)
# img = assembliesShadedView(url,flattenedArrayArray,0.001)
# display(Image(img))

images = []

for ind, matrix in enumerate(arrayArray):
  if ind == 0:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,0.001,"hide","image.jpg")
    im1 = gen_frame("image.jpg")
  else:
    flattenedArrayArray = matrix[0][0:4].tolist()+matrix[1][0:4].tolist()+matrix[2][0:4].tolist()
    assembliesShadedView(url,flattenedArrayArray,0.001,"hide","image.jpg")
    images.append(gen_frame("image.jpg"))

im1.save('OnshapeGIF.gif', save_all=True, loop = 500, append_images=images, disposal=2, duration=400)

In [ ]:
print(arrayArray[0])
np.linalg.det(arrayArray[1])

[[ 8.66025406e-01  4.99999997e-01  6.73058687e-09  0.00000000e+00]
 [-2.50000004e-01  4.33012701e-01  8.66025403e-01  0.00000000e+00]
 [ 4.33012696e-01 -7.50000003e-01  5.00000002e-01  0.00000000e+00]
 [ 1.06514574e-01 -1.85170373e-01  2.23050881e-01  1.00000000e+00]]


0.682690529776203